In [19]:
import vectorbt as vbt
import pandas as pd

# Data loading

In [20]:
def load_prices(tickers, years=10):
    """
    Download adjusted close prices for given tickers
    over a rolling [today - years, today] window.
    """
    end = pd.Timestamp.today().normalize()
    start = end - pd.DateOffset(years=years)

    data = vbt.YFData.download(
        tickers,
        start=start,
        end=end
    )

    prices = data.get("Close")
    return prices.dropna()

In [52]:
qqq = load_prices("QQQ")
xlp = load_prices("XLP")

# tools

In [69]:
def ath_drawdown(price: pd.Series, drawdown_threshold: float = -0.15):
    """
    Agent Tool: ATH Drawdown Detector

    Purpose
    -------
    Identifies periods where an asset is in a significant drawdown
    relative to its running all-time high (ATH).

    Conceptually, this answers:
    "Is this asset currently in a structural stress regime
     compared to its historical peak?"

    Parameters
    ----------
    price : pd.Series
        Price series indexed by datetime.
    drawdown_threshold : float, default -0.15
        Drawdown level that defines a crisis regime.
        Example: -0.15 means '15% below ATH'.

    Returns
    -------
    dict with keys:
        drawdown : pd.Series (float)
            Continuous drawdown value at each timestamp.
            Example: -0.20 means price is 20% below ATH.
        mask : pd.Series (bool)
            True where drawdown <= drawdown_threshold.
            This is the agent-usable regime signal.
        meta : dict
            Metadata describing the threshold used.

    Agent Interpretation
    --------------------
    - drawdown  → information layer (how deep is stress)
    - mask      → decision layer (in crisis or not)
    - meta      → accountability / explainability
    """
    peak = price.cummax()
    drawdown = price / peak - 1
    mask = drawdown <= drawdown_threshold

    meta = {
        "drawdown_threshold": drawdown_threshold
    }

    return {
        "drawdown": drawdown,
        "mask": mask,
        "meta": meta
    }

In [68]:
dd = ath_drawdown(qqq)
dd["mask"].value_counts()

Close
False    1636
True      879
Name: count, dtype: int64

In [70]:
def divergence_event(
    price_a: pd.Series,
    price_b: pd.Series,
    down_threshold_a: float = -0.01,
    up_threshold_b: float = 0.02
):
    """
    Agent Tool: Cross-Asset Divergence Event Detector

    Purpose
    -------
    Detects single-day stress-rotation events where:
    - Asset A experiences a sharp negative return
    - Asset B simultaneously experiences a sharp positive return

    This captures moments of panic rotation or defensive flow.

    Conceptually, this answers:
    "On which days does the market actively flee asset A
     and reward asset B?"

    Parameters
    ----------
    price_a : pd.Series
        Price series of stressed asset (e.g. QQQ).
    price_b : pd.Series
        Price series of potential hedge / refuge asset (e.g. XLP).
    down_threshold_a : float, default -0.01
        Daily return threshold defining stress in asset A.
    up_threshold_b : float, default 0.02
        Daily return threshold defining strength in asset B.

    Returns
    -------
    dict with keys:
        returns_a : pd.Series (float)
            Daily returns of asset A.
        returns_b : pd.Series (float)
            Daily returns of asset B.
        mask : pd.Series (bool)
            True on days where divergence event occurs.
        meta : dict
            Thresholds used to define the event.

    Agent Interpretation
    --------------------
    - returns_* → raw market behavior
    - mask      → rare but high-information stress signal
    - meta      → why the event was triggered
    """
    ret_a = price_a.pct_change()
    ret_b = price_b.pct_change()

    mask = (ret_a <= down_threshold_a) & (ret_b >= up_threshold_b)

    meta = {
        "down_threshold_a": down_threshold_a,
        "up_threshold_b": up_threshold_b
    }

    return {
        "returns_a": ret_a,
        "returns_b": ret_b,
        "mask": mask,
        "meta": meta
    }

In [65]:
div = divergence_event(qqq, xlp)
div["mask"].value_counts()

Close
False    2514
True        1
Name: count, dtype: int64

In [71]:
def rolling_divergence_frequency(
    price_a: pd.Series,
    price_b: pd.Series,
    window: int = 50,
    down_threshold_a: float = -0.01,
    up_threshold_b: float = 0.02,
    freq_threshold: float = 0.15
):
    """
    Agent Tool: Rolling Stress-Correlation Regime Detector

    Purpose
    -------
    Measures how frequently divergence events occur over a rolling window.
    This acts as an event-based correlation metric between asset A stress
    and asset B strength.

    Conceptually, this answers:
    "Is the market *repeatedly* rewarding asset B
     when asset A is under stress?"

    This is NOT linear correlation.
    It is behavioral / regime-based correlation.

    Parameters
    ----------
    price_a : pd.Series
        Price series of stressed asset (e.g. QQQ).
    price_b : pd.Series
        Price series of hedge / refuge asset (e.g. XLP).
    window : int, default 50
        Rolling window size (number of trading days).
    down_threshold_a : float, default -0.01
        Stress threshold for asset A daily returns.
    up_threshold_b : float, default 0.02
        Strength threshold for asset B daily returns.
    freq_threshold : float, default 0.15
        Frequency level that defines a persistent stress regime.
        Example: 0.15 = divergence on 15% of recent days.

    Returns
    -------
    dict with keys:
        frequency : pd.Series (float)
            Rolling frequency of divergence events (0–1).
        regime : pd.Series (bool)
            True where frequency >= freq_threshold.
            This is the agent-usable regime switch.
        events : pd.Series (bool)
            Raw divergence event occurrences.
        meta : dict
            All parameters used to define the regime.

    Agent Interpretation
    --------------------
    - frequency → intensity of stress rotation
    - regime    → persistent risk-off confirmation
    - events    → atomic stress signals
    - meta      → full explainability for decisions
    """
    div = divergence_event(
        price_a,
        price_b,
        down_threshold_a,
        up_threshold_b
    )

    frequency = div["mask"].rolling(window).mean()
    regime = frequency >= freq_threshold

    meta = {
        "window": window,
        "down_threshold_a": down_threshold_a,
        "up_threshold_b": up_threshold_b,
        "freq_threshold": freq_threshold
    }

    return {
        "frequency": frequency,
        "regime": regime,
        "events": div["mask"],
        "meta": meta
    }

In [63]:
stress = rolling_divergence_frequency(qqq, xlp)

stress["frequency"].value_counts()
# stress["regime"].value_counts()

Close
0.00    2416
0.02      50
Name: count, dtype: int64